In [48]:
!pip install transformers

In [49]:
import pandas as pd 
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

In [50]:
df = pd.read_csv("../data/hasil_scraping_demo/fix_scarap/cleaned_data_v2.csv")

In [51]:
df

,created_at,favorite_count,full_text,quote_count,reply_count,retweet_count,hashtags,cleaned_text,cleaned_text_2,tokenized_text,stemmed_text,lemmatized_text
0,Sun Aug 31 14:43:43 +0000 2025,5677,Menyikapi pernyataan Prabowo bersama dengan pi...,54,87,3660,[],menyikapi pernyataan prabowo bersama dengan pi...,menyikapi pernyataan prabowo pimpinan lembaga ...,"['menyikapi', 'pernyataan', 'prabowo', 'pimpin...",sikap nyata prabowo pimpin lembaga negara part...,menyikapi pernyataan prabowo pimpinan lembaga ...
1,Sun Aug 31 10:11:46 +0000 2025,7435,Saudara-saudari sebangsa dan setanah air Negar...,4037,4287,1575,[],saudarasaudari sebangsa dan setanah air negara...,saudarasaudari sebangsa setanah air negara men...,"['saudarasaudari', 'sebangsa', 'setanah', 'air...",saudarasaudari bangsa tanah air negara hormat ...,saudarasaudari sebangsa setanah air negara men...
2,Fri Aug 29 06:27:34 +0000 2025,14161,Ringkasan Pernyataan Presiden: 1. Menuduh demo...,2003,709,14116,[],ringkasan pernyataan presiden menuduh demonst...,ringkasan pernyataan presiden menuduh demonstr...,"['ringkasan', 'pernyataan', 'presiden', 'menud...",ringkas nyata presiden tuduh demonstrasi kemar...,ringkasan pernyataan presiden menuduh demonstr...
3,Sat Aug 30 10:34:24 +0000 2025,4806,Musuh kita bukan fasilitas umum. Bukan perbeda...,34,65,1939,[],musuh kita bukan fasilitas umum bukan perbedaa...,musuh fasilitas perbedaan suku ras agama musuh...,"['musuh', 'fasilitas', 'perbedaan', 'suku', 'r...",musuh fasilitas beda suku ras agama musuh kuas...,musuh fasilitas perbedaan suku ra agama musuh ...
4,Sat Aug 30 06:44:02 +0000 2025,306,R E Q U I E M ● Brutalitas aparat bukan saja p...,1,30,93,[],requiem brutalitas aparat bukan saja problem ...,requiem brutalitas aparat problem struktural k...,"['requiem', 'brutalitas', 'aparat', 'problem',...",requiem brutalitas aparat problem struktural k...,requiem brutalitas aparat problem struktural k...
...,...,...,...,...,...,...,...,...,...,...,...,...
1218,Thu Aug 28 07:51:29 +0000 2025,2541,Hari ini mahasiswa melakukan aksi demo lagi di...,17,41,562,[],hari ini mahasiswa melakukan aksi demo lagi di...,mahasiswa aksi demo didepan gedung dewan perwa...,"['mahasiswa', 'aksi', 'demo', 'didepan', 'gedu...",mahasiswa aksi demo depan gedung dewan wakil r...,mahasiswa aksi demo didepan gedung dewan perwa...
1219,Fri Aug 29 11:33:23 +0000 2025,1557,Demo Bubarkan DPR Evaluasi Pejabat Korup dan T...,15,23,540,[],demo bubarkan dpr evaluasi pejabat korup dan t...,demo bubarkan dewan perwakilan rakyat evaluasi...,"['demo', 'bubarkan', 'dewan', 'perwakilan', 'r...",demo bubar dewan wakil rakyat evaluasi jabat k...,demo bubarkan dewan perwakilan rakyat evaluasi...
1220,Mon Aug 25 05:43:50 +0000 2025,4135,Pantauan Demo di Gedung DPR RI tanggal 25 Agus...,48,49,1120,[],pantauan demo di gedung dpr ri tanggal agustu...,pantauan demo gedung dewan perwakilan rakyat t...,"['pantauan', 'demo', 'gedung', 'dewan', 'perwa...",pantau demo gedung dewan wakil rakyat tanggal ...,pantauan demo gedung dewan perwakilan rakyat t...
1221,Sun Aug 31 22:36:49 +0000 2025,4441,SEPULUH TUNTUTAN RAKYAT UNTUK INDONESIA (SEPUL...,65,299,2135,[],sepuluh tuntutan rakyat untuk indonesia sepult...,sepuluh tuntutan rakyat indonesia sepultura de...,"['sepuluh', 'tuntutan', 'rakyat', 'indonesia',...",puluh tuntut rakyat indonesia sepultura demo s...,sepuluh tuntutan rakyat indonesia sepultura de...


In [52]:
from textblob import TextBlob

In [53]:
data_tweet = list(df['stemmed_text'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(data_tweet):
    analysis = TextBlob(tweet)
    polaritas += analysis.polarity

    if analysis.sentiment.polarity > 0.0:
        total_positif += 1
        status.append('Positif')
    elif analysis.sentiment.polarity == 0.0:
        total_netral += 1
        status.append('Netral')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')

Hasil Analisis Data:
Positif = 70
Netral = 1100
Negatif = 53

Total Data : 1223


In [54]:
df_label = df['stemmed_text'].to_frame()
df_label['klasifikasi_textblob'] = status
df_label

,stemmed_text,klasifikasi_textblob
0,sikap nyata prabowo pimpin lembaga negara part...,Netral
1,saudarasaudari bangsa tanah air negara hormat ...,Netral
2,ringkas nyata presiden tuduh demonstrasi kemar...,Netral
3,musuh fasilitas beda suku ras agama musuh kuas...,Netral
4,requiem brutalitas aparat problem struktural k...,Netral
...,...,...
1218,mahasiswa aksi demo depan gedung dewan wakil r...,Netral
1219,demo bubar dewan wakil rakyat evaluasi jabat k...,Netral
1220,pantau demo gedung dewan wakil rakyat tanggal ...,Positif
1221,puluh tuntut rakyat indonesia sepultura demo s...,Netral


In [55]:
dataset = df['stemmed_text'].to_frame()
dataset = [tuple(x) for x in dataset.to_records(index=False)]

In [56]:
import random
dataset = df_label[['stemmed_text', 'klasifikasi_textblob']].to_records(index=False)
dataset = [tuple(x) for x in dataset]  

set_positif = []
set_negatif = []
set_netral = []

for teks, label in dataset:
    if label == 'Positif':
        set_positif.append((teks, label))
    elif label == 'Negatif':
        set_negatif.append((teks, label))
    else:
        set_netral.append((teks, label))

set_positif = random.sample(set_positif, k=int(len(set_positif) / 2))
set_negatif = random.sample(set_negatif, k=int(len(set_negatif) / 2))
set_netral = random.sample(set_netral, k=int(len(set_netral) / 2))

train = set_positif + set_negatif + set_netral

# Buat list akhir
train_set = list(train)


In [57]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('brown')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to /home/ahmadnafim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ahmadnafim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /home/ahmadnafim/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ahmadnafim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [58]:
!pip install --upgrade textblob


In [59]:
!python -m textblob.download_corpora

[nltk_data] Downloading package brown to /home/ahmadnafim/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/ahmadnafim/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/ahmadnafim/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/ahmadnafim/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to
[nltk_data]     /home/ahmadnafim/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/ahmadnafim/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.


In [60]:
from textblob.classifiers import NaiveBayesClassifier
cl = NaiveBayesClassifier(train_set)
print("Akurasi Test: ", cl.accuracy(dataset))


Akurasi Test:  0.9239574816026165


In [61]:
data_tweet = list(df['stemmed_text'])
polaritas = 0

status = []
total_positif = total_negatif = total_netral = total = 0

for i, tweet in enumerate(data_tweet):
    analysis = TextBlob(tweet, classifier=cl)

    if analysis.classify() == 'Positif':
        total_positif += 1
        status.append('Positif')
    elif analysis.classify() == 0.0:
        total_netral += 1
        status.append('Netral')
    else:
        total_negatif += 1
        status.append('Negatif')

    total += 1

print(f'Hasil Analisis Data:\nPositif = {total_positif}\nNetral = {total_netral}\nNegatif = {total_negatif}')
print(f'\nTotal Data : {total}')

Hasil Analisis Data:
Positif = 21
Netral = 0
Negatif = 1202

Total Data : 1223


In [62]:
df_label['klasikasi_nb'] = status 
df_label['text asli'] = df['cleaned_text_2']
df_label

,stemmed_text,klasifikasi_textblob,klasikasi_nb,text asli
0,sikap nyata prabowo pimpin lembaga negara part...,Netral,Negatif,menyikapi pernyataan prabowo pimpinan lembaga ...
1,saudarasaudari bangsa tanah air negara hormat ...,Netral,Negatif,saudarasaudari sebangsa setanah air negara men...
2,ringkas nyata presiden tuduh demonstrasi kemar...,Netral,Negatif,ringkasan pernyataan presiden menuduh demonstr...
3,musuh fasilitas beda suku ras agama musuh kuas...,Netral,Negatif,musuh fasilitas perbedaan suku ras agama musuh...
4,requiem brutalitas aparat problem struktural k...,Netral,Negatif,requiem brutalitas aparat problem struktural k...
...,...,...,...,...
1218,mahasiswa aksi demo depan gedung dewan wakil r...,Netral,Negatif,mahasiswa aksi demo didepan gedung dewan perwa...
1219,demo bubar dewan wakil rakyat evaluasi jabat k...,Netral,Negatif,demo bubarkan dewan perwakilan rakyat evaluasi...
1220,pantau demo gedung dewan wakil rakyat tanggal ...,Positif,Negatif,pantauan demo gedung dewan perwakilan rakyat t...
1221,puluh tuntut rakyat indonesia sepultura demo s...,Netral,Negatif,sepuluh tuntutan rakyat indonesia sepultura de...


In [63]:
data_eval = [tuple(x) for x in df_label.to_records(index=False)]

for i in data_eval:
    print(f'Text: {i[0]}\nTextBlob: {i[1]}\nNaiveBayes: {i[2]}\n')


Text: sikap nyata prabowo pimpin lembaga negara partai politik koalisi masyarakat sipil indonesia prabowo gagal paham respons dinamika sosial politik ekonomi timbul marah publik akhirakhir demonstrasi
TextBlob: Netral
NaiveBayes: Negatif

Text: saudarasaudari bangsa tanah air negara hormat bebas sampai dapat aspirasi murni masyarakat tugas salah polri periksa cepat transparan buka publik pimpin dewan wakil rakyat
TextBlob: Netral
NaiveBayes: Negatif

Text: ringkas nyata presiden tuduh demonstrasi kemarin arah arah tindak anarkis keras bunuh demonstran mundur tabrak lindas tangkap presiden maaf bertanggungjawab bunuh
TextBlob: Netral
NaiveBayes: Negatif

Text: musuh fasilitas beda suku ras agama musuh kuasa arogan tindas rakyat larut konflik salah rakyat salah negara hadir rakyat protes salah media
TextBlob: Netral
NaiveBayes: Negatif

Text: requiem brutalitas aparat problem struktural kanker kultural republik entitas suka banget ngaku bhw politik politik negara dengar mulia telinga war

In [65]:
positif_texts = df_label[df_label['klasikasi_nb'] == 'Positif']
for idx, row in positif_texts.iterrows():
    print(f"Text: {row['text asli']}\nNB Prediction: {row['klasikasi_nb']}\n")

Text: polisi indo muslim aja ngingetin salah alasan abadi neraka membunuh muslim but you will never care its okay kepastian hadapi kematian youll see
NB Prediction: Positif

Text: rakyat polisi rakyat salah polisi salah memasuki era devide at impera dimana warga sipil dibenturkan aparat pejabat elit pemegang kekuasaan extraordinary mengacuhkan prosedural cepat memulihkan
NB Prediction: Positif

Text: temanteman stay safe ya besok kantor kuliah please kasih kabar live location minimal salah kerabat semoga kecuali prabowo gibran polisi dewan perwakilan rakyat lindungan tuhan
NB Prediction: Positif

Text: polisi indo muslim aja ngingetin salah alasan abadi neraka membunuh muslim but you will never care its okay kepastian hadapi kematian youll see
NB Prediction: Positif

Text: tolong temen temen semuanyaa stay safe yaaa dapet info sepupu mahasiswi ui polisi nyamar sekitaran stasiuntransum geledahin tas orang orang cari mahasiswa pulang demo ngikutin sampe kosan
NB Prediction: Positif

Text